В [статье](https://arxiv.org/pdf/1506.06297.pdf) говорится, что использование наркотиков можно разделить на плеяды. Это три плеяды: героиновая плеяда, плеяда экстази и плеяда бензодиазепинов по отношению к десятилетию, годам, разделение пользователей и не пользователей по месяцам и неделям. Можно свести задачу к multilabel классификации, разбив на плеяды получим 3 класса.




In [1]:
# импортируем нужные нам библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, log_loss, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from model import forest_importances, model_grid, cross_val_multilabel

np.random.seed(100)

In [2]:
df = pd.read_csv('../data/data_preprocessed.csv')

In [3]:
df.tail()

,Age,Gender,Educ,Cntry,Ethn,NS,ES,OS,AS,CS,...,Ecstasy,Heroin,Ketamine,LegalH,LSD,Meth,Shrooms,Nicotine,Semer,VSA
1880,18-24,0,Left schl at 17,USA,White,-1.19430,1.74091,1.88511,0.76096,-1.13788,...,CL0,CL0,CL0,CL3,CL3,CL0,CL0,CL0,CL0,CL5
1881,18-24,1,Left schl at 17,USA,White,-0.24649,1.74091,0.58331,0.76096,-1.51840,...,CL2,CL0,CL0,CL3,CL5,CL4,CL4,CL5,CL0,CL0
1882,25-34,0,univ degree,USA,White,1.13281,-1.37639,-1.27553,-1.77200,-1.38502,...,CL4,CL0,CL2,CL0,CL2,CL0,CL2,CL6,CL0,CL0
1883,18-24,0,Left schl at 17,USA,White,0.91093,-1.92173,0.29338,-1.62090,-2.57309,...,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL4,CL0,CL0
1884,18-24,1,Left schl at 17,IRE,White,-0.46725,2.12700,1.65653,1.11406,0.41594,...,CL3,CL0,CL0,CL3,CL3,CL0,CL3,CL6,CL0,CL2


# **Multilabel classification** 


Preprocessing data

In [4]:
def classes2binary(a):
  '''Если класс равен CL6, CL5, CL4 то мы делаем метку 1,
     а если CL0, CL1, CL2, CL3 то 0.'''
  if ((a == 'CL6') or (a == 'CL5') or (a == 'CL4')):
        return 1
  else:
        return 0

In [5]:
for i in df.iloc[:, 12:].columns:
  df[i] = df[i].map(classes2binary)

In [6]:
df.head()

,Age,Gender,Educ,Cntry,Ethn,NS,ES,OS,AS,CS,...,Ecstasy,Heroin,Ketamine,LegalH,LSD,Meth,Shrooms,Nicotine,Semer,VSA
0,35-44,0,prof cert/diploma,UK,Mixed-White/Asian,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,...,0,0,0,0,0,0,0,0,0,0
1,25-34,1,PhD,UK,White,-0.67825,1.93886,1.43533,0.76096,-0.14277,...,1,0,0,0,0,0,0,1,0,0
2,35-44,1,prof cert/diploma,UK,White,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,...,0,0,0,0,0,0,0,0,0,0
3,18-24,0,Ms degree,UK,White,-0.14882,-0.80615,-0.01928,0.59042,0.58489,...,0,0,0,0,0,0,0,0,0,0
4,35-44,0,PhD,UK,White,0.73545,-1.63340,-0.45174,-0.30172,1.30612,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Преобразуем категориальные переменные и колонки, которые были изначально тоже была категориальными('Alcohol', 'Caffeine',	'Choco',	'Nicotine')
# с помощью метода One Hot Encoding
multi_df = pd.get_dummies(df, columns = 
                          list(df.select_dtypes(include='object').columns))

In [8]:
multi_df.head()

,Gender,NS,ES,OS,AS,CS,Imp,SS,Alcohol,Amphet,...,Cntry_Other,Cntry_UK,Cntry_USA,Ethn_Asian,Ethn_Black,Ethn_Mixed-Black/Asian,Ethn_Mixed-White/Asian,Ethn_Mixed-White/Black,Ethn_Other,Ethn_White
0,0,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,1,0,...,0,1,0,0,0,0,1,0,0,0
1,1,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,1,0,...,0,1,0,0,0,0,0,0,0,1
2,1,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,1,0,...,0,1,0,0,0,0,0,0,0,1
3,0,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,1,0,...,0,1,0,0,0,0,0,0,0,1
4,0,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,1,0,...,0,1,0,0,0,0,0,0,0,1


1. The heroin pleiad includes crack, cocaine, methadone, and heroin;
2. The ecstasy pleiad consists of amphetamines, cannabis, cocaine, ketamine, LSD,
magic mushrooms, legal highs, and ecstasy;
3. The benzodiazepines pleiad contains methadone, amphetamines, cocaine, and
benzodiazepines.

In [9]:
multi_df['heroinPl'] = multi_df.apply(lambda x: int((x['Coke'] + x['Crack'] + x['Heroin'] + x['Meth'])>0), axis = 1)
multi_df['ecstasyPl'] = multi_df.apply(lambda x: int((x['Amphet']  + x['Cannabis'] + x['Coke']  + x['Ecstasy'] + x['Ketamine'] + x['LSD'] + x['Meth'] + x['Shrooms']+x['LegalH'] )>0), axis = 1)
multi_df['benzoPl'] = multi_df.apply(lambda x: int((x['Amphet'] + x['Coke'] + x['Benzos'] + x['Meth'])>0), axis = 1)

In [10]:
# Удаляем данные которые использовали для определения плеяд чтобы не было data leakage
multi_df.drop(['Amphet','Benzos', 'Cannabis', 'Coke', 'Crack', 'Heroin', 'Meth', 'Ecstasy', 'Ketamine', 'LSD', 'Shrooms', 'LegalH'], axis=1, inplace=True)

Берем только значения где пристутвиет плеяды, а где нет не берем.

In [11]:
multi_df = multi_df[(multi_df["heroinPl"] == 1
          ) | (multi_df["ecstasyPl"] == 1
          ) | (multi_df["benzoPl"] == 1)]
          
multi_df.shape

(994, 46)

In [12]:
multi_df[['heroinPl', 'ecstasyPl', 'benzoPl']].head()

,heroinPl,ecstasyPl,benzoPl
1,0,1,0
11,0,1,0
12,0,0,1
17,0,1,0
19,1,1,1


In [13]:
multi_df.head()

,Gender,NS,ES,OS,AS,CS,Imp,SS,Alcohol,Amyl,...,Ethn_Asian,Ethn_Black,Ethn_Mixed-Black/Asian,Ethn_Mixed-White/Asian,Ethn_Mixed-White/Black,Ethn_Other,Ethn_White,heroinPl,ecstasyPl,benzoPl
1,1,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,1,0,...,0,0,0,0,0,0,1,0,1,0
11,1,-1.32828,0.00332,0.14143,-1.92595,-0.52745,0.52975,1.22470,1,0,...,0,0,0,0,0,0,1,0,1,0
12,0,2.28554,0.16767,0.44585,-1.62090,-0.78155,1.29221,0.07987,1,0,...,0,0,0,0,0,0,1,0,0,1
17,1,0.52135,-1.23177,-0.31776,-0.45321,-1.38502,-1.37983,-0.84637,1,0,...,0,0,0,0,0,0,1,0,1,0
19,1,-0.34799,-1.76250,-2.39883,-1.92595,0.75830,-1.37983,-2.07848,1,0,...,0,0,0,0,0,0,1,1,1,1


In [14]:
# Удаляем метки из обучающей части и коррелирующее значение с SS Imp  
multi_train, y = multi_df.drop(['heroinPl', 'ecstasyPl', 'benzoPl', 'Imp'], axis=1), multi_df[['heroinPl', 'ecstasyPl', 'benzoPl']]

In [15]:
x_train, x_test, y_train, y_test = train_test_split(multi_train, y, test_size=.2, random_state=13)

In [16]:
x_train.shape, y_train.shape

((795, 42), (795, 3))

In [17]:
model = MultiOutputClassifier(RandomForestClassifier())
model.fit(x_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [18]:
model.estimators_

[RandomForestClassifier(), RandomForestClassifier(), RandomForestClassifier()]

In [19]:
y_pred = np.transpose([pred[:, 1] for pred in model.predict_proba(x_test)])

In [20]:
roc_auc_score(y_test, y_pred, average=None)

array([0.64911406, 0.81900585, 0.67686035])

## Отбор признаков для MultiLabel 

In [21]:
feat_heroinPl, feat_ecstasyPl, feat_benzoPl = forest_importances(model.estimators_[0], x_train),\
                                              forest_importances(model.estimators_[1], x_train),\
                                              forest_importances(model.estimators_[2], x_train)

Смотрим какие фичи менее важны и удаляем их из данных

In [22]:
feat_heroinPl.tail()

,Feature,Importance
37,Semer,0.001872
38,Age_65+,0.001189
39,Ethn_Mixed-Black/Asian,0.000859
40,Ethn_Asian,0.000572
41,Cntry_NZ,0.000549


In [23]:
feat_ecstasyPl.tail()

,Feature,Importance
37,Cntry_NZ,2.267268e-04
38,Cntry_IRE,2.079371e-04
39,Ethn_Asian,8.288388e-07
40,Semer,0.000000e+00
41,Ethn_Mixed-Black/Asian,0.000000e+00


In [24]:
feat_benzoPl.tail()

,Feature,Importance
37,Age_65+,0.001220
38,Ethn_Black,0.001135
39,Cntry_NZ,0.000953
40,Ethn_Asian,0.000819
41,Semer,0.000416


In [25]:
# Удаляем столбцы 'Semer', 'Ethn_Mixed-Black/Asian', 'Ethn_Mixed-White/Black', 'Ethn_Black'
x_train, x_test, y_train, y_test = train_test_split(multi_train.drop(['Semer', 'Ethn_Mixed-Black/Asian', 'Ethn_Mixed-White/Black', 'Ethn_Black'], axis=1),\
                                                    y, test_size=.2, random_state=13)

In [26]:
# Обучаем завново модель и считаем roc-auc
model = MultiOutputClassifier(RandomForestClassifier())
model.fit(x_train, y_train)
y_pred = np.transpose([pred[:, 1] for pred in model.predict_proba(x_test)])
roc_auc_score(y_test, y_pred, average=None)

array([0.64928018, 0.87573099, 0.63807339])

Получаем 3 ROC-AUC значения для каждой метки свой.

## Обучение моделей для MultiLabel классификатора

In [27]:
# Смотрим на roc-auc без настроки моделей
classifiers = [
    MultiOutputClassifier(RandomForestClassifier()),
    MultiOutputClassifier(LGBMClassifier()),
    MultiOutputClassifier(CatBoostClassifier(verbose=False))
    ]

for classifier in classifiers:
    steps = [
        ('clf', classifier),
    ]
    pipeline = Pipeline(steps)
    pipeline.fit(x_train, y_train)
    y_pred = np.transpose([pred[:, 1] for pred in pipeline.predict_proba(x_test)])
    print('model:', classifier, '\nroc-auc: {} \n'.format(roc_auc_score(y_test, y_pred, average=None)))

model: MultiOutputClassifier(estimator=RandomForestClassifier()) 
roc-auc: [0.63803987 0.75087719 0.64169215] 

model: MultiOutputClassifier(estimator=LGBMClassifier()) 
roc-auc: [0.59723145 0.80584795 0.62711519] 

model: MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7f7999e72ca0>) 
roc-auc: [0.63023256 0.89415205 0.66167176] 



## **Random Forest**

In [28]:
%%time
params = {
          'n_estimators': range(31, 255, 16),
          'min_samples_split': [4,5,6],
          'max_depth': [50, 60]
         }
         
multi_rf = model_grid(RandomForestClassifier(), params, x_train, y_train, scoring='roc_auc', multi_label=True)

CPU times: user 5.34 s, sys: 297 ms, total: 5.64 s
Wall time: 3min 45s


In [29]:
multi_rf.estimators_[0].best_params_,\
multi_rf.estimators_[1].best_params_,\
multi_rf.estimators_[2].best_params_

({'max_depth': 60, 'min_samples_split': 5, 'n_estimators': 127},
 {'max_depth': 50, 'min_samples_split': 6, 'n_estimators': 31},
 {'max_depth': 60, 'min_samples_split': 5, 'n_estimators': 127})

In [30]:
multi_rf.estimators_[0].best_params_

{'max_depth': 60, 'min_samples_split': 5, 'n_estimators': 127}

In [31]:
model_rf = MultiOutputClassifier(RandomForestClassifier(**multi_rf.estimators_[0].best_params_)).fit(x_train, y_train)
y_pred = np.transpose([pred[:,1] for pred in model_rf.predict_proba(x_test)])
roc_auc_score(y_test, y_pred, average=None)

array([0.64186047, 0.81169591, 0.65616718])

## **LGBM**

In [32]:
params = {
    'max_depth': range(31, 64, 8),
    'learning_rate': np.linspace(0.001, 0.05, 5),
    'n_estimators': range(63, 255, 32),
}

multi_lgbm = model_grid(LGBMClassifier(), params, x_train, y_train, scoring='roc_auc', multi_label=True)

In [33]:
multi_lgbm.estimators_[0].best_params_,\
multi_lgbm.estimators_[1].best_params_,\
multi_lgbm.estimators_[2].best_params_

({'learning_rate': 0.025500000000000002, 'max_depth': 31, 'n_estimators': 223},
 {'learning_rate': 0.013250000000000001, 'max_depth': 31, 'n_estimators': 95},
 {'learning_rate': 0.05, 'max_depth': 31, 'n_estimators': 95})

In [34]:
lgbm = MultiOutputClassifier(LGBMClassifier(**multi_lgbm.estimators_[1].best_params_)).fit(x_train, y_train)
y_pred = np.transpose([pred[:,1] for pred in lgbm.predict_proba(x_test)])
roc_auc_score(y_test, y_pred, average=None)

array([0.59878184, 0.83274854, 0.63802243])

## **Catboost**

In [35]:
params = {
    'iterations': range(400, 1300, 400),
    'depth': range(4, 11, 2)
}

multi_cat = model_grid(CatBoostClassifier(verbose=False), params, x_train, y_train, scoring='roc_auc', multi_label=True)

In [36]:
multi_cat.estimators_[0].best_params_,\
multi_cat.estimators_[1].best_params_,\
multi_cat.estimators_[2].best_params_

({'depth': 8, 'iterations': 1200},
 {'depth': 4, 'iterations': 1200},
 {'depth': 10, 'iterations': 400})

In [37]:
cat = MultiOutputClassifier(CatBoostClassifier(**multi_cat.estimators_[1].best_params_, verbose=False)).fit(x_train, y_train)
y_pred = np.transpose([pred[:,1] for pred in cat.predict_proba(x_test)])
roc_auc_score(y_test, y_pred, average=None)

array([0.62978959, 0.88538012, 0.64475025])

## Cross-validation multi-label classifier

Так как у целевой переменной содержится 3 метки.
При попытке разбить на KFold выдается ошибка, что целевая переменная должна быть с одним измерением. Реализуем функцию, которая будет разбивать на фолды и возвращать значения полученных roc-auc.

In [38]:
cross_val_multilabel(multi_rf, x_train, y_train, 5)

[array([0.60686234, 0.7       , 0.61651871]),
 array([0.61669759, 0.82698675, 0.71740506]),
 array([0.62331081, 0.68058968, 0.63904762]),
 array([0.5929557 , 0.76490066, 0.67575758]),
 array([0.66330275, 0.44880174, 0.66455696])]

In [39]:
cross_val_multilabel(multi_lgbm, x_train, y_train, 5)

[array([0.58164531, 0.70649351, 0.64695625]),
 array([0.67643785, 0.8009106 , 0.67626582]),
 array([0.64977477, 0.64158477, 0.64222222]),
 array([0.61092956, 0.54304636, 0.65117845]),
 array([0.62972477, 0.5087146 , 0.62642405])]

In [40]:
cross_val_multilabel(multi_cat, x_train, y_train, 5)

[array([0.59838776, 0.76493506, 0.61192137]),
 array([0.66252319, 0.7955298 , 0.71471519]),
 array([0.60660661, 0.71990172, 0.65412698]),
 array([0.61819172, 0.75331126, 0.64292929]),
 array([0.69688073, 0.52287582, 0.64636076])]

Можем сделать вывод, что методы основанные на град.бустиге дают значительно высокий результат.